In [1]:
import numpy as np
import pandas as pd

In [38]:
train_raw = pd.read_csv('/kaggle/input/raw-data/train_data_converted_to_.csv')
test_raw = pd.read_csv('/kaggle/input/raw-data/test_data_converted_to_.csv')

In [40]:
col_order = train_raw.columns.to_list()
col_order.remove('y')
test_raw = test_raw[col_order]

In [41]:
label = pd.read_csv('/kaggle/input/raw-data/mask_to_actual_labeling.csv')

In [42]:
zero_or_nan_cols = train_raw.columns[
    ((train_raw == 0) | train_raw.isna()).all()
].tolist()
train_raw.drop(columns=zero_or_nan_cols,inplace=True)
test_raw.drop(columns=zero_or_nan_cols,inplace=True)

In [43]:
columns_with_only_1_or_nan = train_raw.columns[
    ((train_raw == 1) | train_raw.isna()).all()
].tolist()
for col in columns_with_only_1_or_nan:
    train_raw[col] = train_raw[col].fillna(0)
    test_raw[col] = test_raw[col].fillna(0)

In [44]:
methods = pd.read_csv('/kaggle/input/raw-data/Imputation_methods.csv')

In [45]:
methods['imputed with'].unique()

array(['full', 'mode', 'zero', 'deleted', 'mean', 'recalc', 'all zeros',
       'mean_of_that_offer', 'mode_of_that_offer'], dtype=object)

In [46]:
for col in train_raw.columns:
    if col not in methods['masked_column'].to_list():
        print(col)

Unnamed: 0
f218


In [47]:
train_raw.drop(columns=['Unnamed: 0','f218'],inplace=True)
test_raw.drop(columns=['Unnamed: 0','f218'],inplace=True)

In [51]:
num_cols = label[label['Type']=='Numerical']['masked_column'].to_list()
copy = num_cols.copy()
for col in copy:
    if col not in train_raw.columns:
        num_cols.remove(col)
for col in num_cols:
    method = methods[methods['masked_column']==col]['imputed with'].to_list()[0]
    if method=='mode':
        train_raw[col] = train_raw[col].fillna(train_raw[col].mode()[0])
        test_raw[col] = test_raw[col].fillna(test_raw[col].mode()[0])
    elif method=='mean':
        train_raw[col] = train_raw[col].fillna(train_raw[col].mean())
        test_raw[col] = test_raw[col].fillna(test_raw[col].mean())
    elif method=='zero':
        train_raw[col] = train_raw[col].fillna(0)
        test_raw[col] = test_raw[col].fillna(0)
    else:
        print(col," : ",method)

id4  :  full
id5  :  full
f18  :  deleted
f22  :  deleted
f33  :  deleted
f34  :  deleted
f36  :  deleted
f37  :  deleted
f187  :  recalc
f188  :  recalc
f189  :  recalc
f190  :  recalc
f191  :  recalc
f192  :  recalc
f193  :  recalc
f194  :  recalc
f195  :  recalc
f196  :  recalc
f197  :  recalc
f347  :  mean_of_that_offer
f348  :  mode_of_that_offer


In [52]:
cols = ['f18','f22','f33','f34','f36','f37']
for col in cols:
    print(train_raw[col].value_counts())

f18
1.0    689
0.0      7
Name: count, dtype: int64
f22
2.0       46383
1.0       41374
4.0       33285
3.0       28870
6.0       28111
          ...  
103.0         1
3645.0        1
470.0         1
343.0         1
143.0         1
Name: count, Length: 245, dtype: int64
f33
1.0      18923
2.0       5708
3.0       1401
4.0       1143
5.0        435
6.0        313
8.0        228
9.0        134
14.0        87
7.0         50
11.0        16
19.0         9
10.0         8
12.0         5
13.0         4
180.0        2
117.0        1
Name: count, dtype: int64
f34
1.0    393
2.0      4
3.0      4
7.0      2
4.0      1
Name: count, dtype: int64
f36
61.0     926
52.0     829
86.0     718
58.0     565
103.0    534
        ... 
305.0      1
330.0      1
586.0      1
430.0      1
582.0      1
Name: count, Length: 483, dtype: int64
f37
1.0     768
2.0     263
5.0     104
3.0      49
4.0      20
6.0       9
8.0       9
17.0      7
11.0      4
21.0      4
18.0      3
7.0       3
9.0       2
Name: count, 

In [53]:
train_raw['f18'] = train_raw['f18'].fillna(0)
train_raw['f22'] = train_raw['f22'].fillna(0)
train_raw['f33'] = train_raw['f33'].fillna(0)
train_raw['f34'] = train_raw['f34'].fillna(0)
train_raw['f36'] = train_raw['f36'].fillna(0)
train_raw['f37'] = train_raw['f37'].fillna(0)

test_raw['f18'] = test_raw['f18'].fillna(0)
test_raw['f22'] = test_raw['f22'].fillna(0)
test_raw['f33'] = test_raw['f33'].fillna(0)
test_raw['f34'] = test_raw['f34'].fillna(0)
test_raw['f36'] = test_raw['f36'].fillna(0)
test_raw['f37'] = test_raw['f37'].fillna(0)

In [54]:
cols=[]
for i in range(187,198):
    col = 'f'+str(i)
    cols.append(col)
train_raw.drop(columns=cols,inplace=True)
test_raw.drop(columns=cols,inplace=True)

In [55]:
cols2=[]
for i in range(174,185):
    col='f'+str(i)
    cols2.append(col)

In [57]:
train_raw['dummy']=0
test_raw['dummy']=0
for col in cols2:
    train_raw['dummy']+=train_raw[col]
    test_raw['dummy']+=test_raw[col]
for col1,col2 in zip(cols,cols2):
    train_raw[col1] = train_raw[col2]/train_raw['dummy']
    test_raw[col1] = test_raw[col2]/test_raw['dummy']
train_raw.drop(columns=['dummy'],inplace=True)
test_raw.drop(columns=['dummy'],inplace=True)

/tmp/ipykernel_36/4203722678.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_raw['dummy']=0
/tmp/ipykernel_36/4203722678.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_raw[col1] = train_raw[col2]/train_raw['dummy']
/tmp/ipykernel_36/4203722678.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = f

In [58]:
train_raw = train_raw.copy()
test_raw = test_raw.copy()

In [59]:
for col in cols:
    train_raw[col] = train_raw[col].fillna(0)
    test_raw[col] = test_raw[col].fillna(0)

In [60]:
train_raw['f347'] = train_raw['f347'].fillna(train_raw.groupby('id3')['f347'].transform('mean'))
train_raw['f348'] = train_raw['f348'].fillna(train_raw.groupby('id3')['f348'].transform('mean'))
train_raw['f347'] = train_raw['f347'].fillna(0)
train_raw['f348'] = train_raw['f348'].fillna(0)

test_raw['f347'] = test_raw['f347'].fillna(test_raw.groupby('id3')['f347'].transform('mean'))
test_raw['f348'] = test_raw['f348'].fillna(test_raw.groupby('id3')['f348'].transform('mean'))
test_raw['f347'] = test_raw['f347'].fillna(0)
test_raw['f348'] = test_raw['f348'].fillna(0)

In [61]:
onehot_cols = label[label['Type']=='One hot encoded']['masked_column'].to_list()
copy = onehot_cols.copy()
for col in copy:
    if col not in train_raw.columns:
        onehot_cols.remove(col)
for col in onehot_cols:
    train_raw[col] = train_raw[col].fillna(0)
    test_raw[col] = test_raw[col].fillna(0)

In [62]:
import pandas.api.types as ptype
cat_cols = label[label['Type']=='Categorical']['masked_column'].to_list()
for col in cat_cols:
    if ptype.is_object_dtype(train_raw[col]):
        train_raw[col] = train_raw[col].fillna("__missing__")
        test_raw[col] = test_raw[col].fillna("__missing__")
    elif ptype.is_numeric_dtype(train_raw[col]):
        train_raw[col] = train_raw[col].fillna(-1)
        test_raw[col] = test_raw[col].fillna(-1)
    else:
        print(col)

In [63]:
train_raw.isnull().any().any()

False

In [64]:
test_raw.isnull().any().any()

False

In [66]:
train_raw['id4'] = pd.to_datetime(train_raw['id4'])
test_raw['id4'] = pd.to_datetime(test_raw['id4'])

train_raw['year'] = train_raw['id4'].dt.year
train_raw['month'] = train_raw['id4'].dt.month
train_raw['day'] = train_raw['id4'].dt.day
train_raw['dayofweek'] = train_raw['id4'].dt.dayofweek # 0=Monday and 6=Sunday
train_raw['weekofyear'] = train_raw['id4'].dt.isocalendar().week
train_raw['quarter'] = train_raw['id4'].dt.quarter
train_raw['hour'] = train_raw['id4'].dt.hour
train_raw['minute'] = train_raw['id4'].dt.minute
train_raw['second'] = train_raw['id4'].dt.second
train_raw['is_month_start'] = train_raw['id4'].dt.is_month_start
train_raw['is_month_end'] = train_raw['id4'].dt.is_month_end
train_raw['is_weekend'] = train_raw['id4'].dt.dayofweek >= 5

test_raw['year'] = test_raw['id4'].dt.year
test_raw['month'] = test_raw['id4'].dt.month
test_raw['day'] = test_raw['id4'].dt.day
test_raw['dayofweek'] = test_raw['id4'].dt.dayofweek # 0=Monday and 6=Sunday
test_raw['weekofyear'] = test_raw['id4'].dt.isocalendar().week
test_raw['quarter'] = test_raw['id4'].dt.quarter
test_raw['hour'] = test_raw['id4'].dt.hour
test_raw['minute'] = test_raw['id4'].dt.minute
test_raw['second'] = test_raw['id4'].dt.second
test_raw['is_month_start'] = test_raw['id4'].dt.is_month_start
test_raw['is_month_end'] = test_raw['id4'].dt.is_month_end
test_raw['is_weekend'] = test_raw['id4'].dt.dayofweek >= 5

train_raw.drop(columns=['id4'],inplace=True)
test_raw.drop(columns=['id4'],inplace=True)

In [67]:
train_raw.shape

(770164, 321)

In [68]:
test_raw.shape

(369301, 320)

In [69]:
mask=list(train_raw['f29']>train_raw['f28'])
train_raw.loc[mask,'f29']=0
train_raw.loc[mask,'f28']=0

mask=list(test_raw['f29']>test_raw['f28'])
test_raw.loc[mask,'f29']=0
test_raw.loc[mask,'f28']=0

In [70]:
mask=list(train_raw['f31']>train_raw['f30'])
train_raw.loc[mask,'f30']=0
train_raw.loc[mask,'f31']=0

mask=list(test_raw['f31']>test_raw['f30'])
test_raw.loc[mask,'f30']=0
test_raw.loc[mask,'f31']=0

In [71]:
train_raw.to_csv('train_cleaned.csv',index=False)
test_raw.to_csv('test_cleaned.csv',index=False)